In [ ]:
# install necessary python libraries 
%pip install python-bitcoinrpc
%pip install paramiko

In [116]:
# read credentials from file 
import pandas as pd

credentials = pd.read_json('credentials.json')

In [117]:
# establish ssh connection to the host 
import paramiko 
host = "artemis.uni.ma"
port = 22
username = credentials['user'][0]
password = credentials['pwd'][0]

ssh = paramiko.SSHClient()
ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
ssh.connect(host, port, username, password)

In [ ]:
stdin, stdout, stderr = ssh.exec_command("bitcoin-cli getbestblockhash") # returns the header hash of the most recent block on the best block chain.
print(stdout.read().decode("utf-8")) 

# Command: bitcoin-cli getblock [blockhash] 1
Get blocks via the command `bitcoin-cli getblock [blockhash] 1`. The output contains information such as the transactions belonging to this block as well as the `previousblockhash` and `nextblockhash`. Via these attributes blocks can be retrieved in an ordered fashion and we have the neccessary information about the `PRECEDES/BELONGS_TO` relationship. 
In addition, this command returns the `blockheight` and `blocktime` (e.g., 1619858709) and an array of `txids` which are associated/ stored with the block. 

In [ ]:
stdin, stdout, stderr = ssh.exec_command("bitcoin-cli getblock 00000000000000000008c69128a76fb9888164db31dace993cf2a6b8a574664a 1")
print(stdout.read().decode("utf-8")) 

# Command: bitcoin-cli getrawtransaction [txid] true
Number of objects contained in `vin` equals `inDegree`(?)
Number of objects contained in `vout` equals `outDegree`(?) 
returns info about which block in belongs => `BELONG_TO` relationship


In [97]:
stdin, stdout, stderr = ssh.exec_command("bitcoin-cli getrawtransaction 3511d57b5d64df0c1ba7c8b9d16ecdad7ea032e31003ac76abb9118e50ccea67 true") 

# parse json output
import json
rawtx = json.loads(stdout.read().decode("utf-8"))

print("Indegree: ", len(rawtx['vin']))
print("Outdegree: ", len(rawtx['vout']))

inSum = 0 
for vin in rawtx['vin']:
    print(vin['txid'])
    inSum += vin['vout']
    
print("InSum: ", inSum)
    
outSum = 0
for v in rawtx['vout']:
    outSum += v['value']
    for a in v['scriptPubKey']['addresses']:
        print(a)
    
    
print("OutSum: ", outSum)

Indegree:  1
Outdegree:  2
946b547d6103956ef9d93f3142c0a190d7ca58f505bcd21e9bda833ebecebb30
InSum:  0
1FabnFuatDMP15qmG8B66sw4uDjjUumW2b
12DsinE3HnKj3cqaZZ523QfTmaoNy6jo4S
OutSum:  28.07


In [37]:
stdin, stdout, stderr = ssh.exec_command("bitcoin-cli getrawtransaction 3511d57b5d64df0c1ba7c8b9d16ecdad7ea032e31003ac76abb9118e50ccea67 true")
print(stdout.read().decode("utf-8")) 


{
  "txid": "3511d57b5d64df0c1ba7c8b9d16ecdad7ea032e31003ac76abb9118e50ccea67",
  "hash": "3511d57b5d64df0c1ba7c8b9d16ecdad7ea032e31003ac76abb9118e50ccea67",
  "version": 1,
  "size": 259,
  "vsize": 259,
  "weight": 1036,
  "locktime": 0,
  "vin": [
    {
      "txid": "946b547d6103956ef9d93f3142c0a190d7ca58f505bcd21e9bda833ebecebb30",
      "vout": 0,
      "scriptSig": {
        "asm": "3046022100e51d22571a1ca1be621fa4cd0b209fc27d776ac26142c1ff58c280f27da48173022100a9ee6a6ecd912419aa4b1b7faebdb2ee92034b6a304c036c8d4512cda407f95e[ALL] 0459eb48ac34283a36c04ae930b9d7486a2d9138ad2d30549a6e899f2eb3bce993f5dc9e38bb1759d9b811cbebdb5b45f681099eb626b3ad9a9f075db5869bdfde",
        "hex": "493046022100e51d22571a1ca1be621fa4cd0b209fc27d776ac26142c1ff58c280f27da48173022100a9ee6a6ecd912419aa4b1b7faebdb2ee92034b6a304c036c8d4512cda407f95e01410459eb48ac34283a36c04ae930b9d7486a2d9138ad2d30549a6e899f2eb3bce993f5dc9e38bb1759d9b811cbebdb5b45f681099eb626b3ad9a9f075db5869bdfde"
      },
      "sequence":

In [ ]:
# address - tx.outputs[o].addresses[0].address
# blocks - [block_hash, block_height, block_timestamp]
# transaction - txid, block_date, inDegree, outDegree

In [ ]:
# before relationship - ([previous_block_hash, block_hash, 'PRECEDES']) OK
# belongs relationship - txid, block_hash, 'BELONGS_TO' OK
# receives relationship - [tx_id, val, o, addr, 'RECEIVES'] o= index array OK
# sends relationship - ([in_address, in_value, tx_id, 'SENDS'])

# Complete Process
continuously monitor the bestblockhash - returns the tip of the blockchain

In [126]:
# get best blockhash - returns the header hash of the most recent block on the best block chain
stdin, stdout, stderr = ssh.exec_command("bitcoin-cli getbestblockhash")
bestblockhash = stdout.read().decode("utf-8")
print(bestblockhash) 

0000000000000000000a81ab936f106b98c6b3b6139a959dc2e2c824ece21676



In [120]:
# get blockinfo 
command = "bitcoin-cli getblock " + bestblockhash + " 2"
#stdin, stdout, stderr = ssh.exec_command("bitcoin-cli getblock 00000000000000000008bb27dd339ad1b2534f9060907a2be6925bb375395ba2 1")
stdin, stdout, stderr = ssh.exec_command(command)
block = json.loads(stdout.read().decode("utf-8"))

# before relationship - ([previous_block_hash, block_hash, 'PRECEDES']) OK
previousblockhash = block['previousblockhash']
nextblockhash = block['nextblockhash']

print("previousblockhash: ", previousblockhash)
print("nextblockhash: ", nextblockhash)

# blocks - [block_hash, block_height, block_timestamp]
from datetime import datetime, timezone
ts_epoch = block['time']
block_timestamp = datetime.fromtimestamp(ts_epoch)
block_date = block_timestamp.strftime('%Y-%m-%d')
block_hash = block['hash']
block_height = block['height']

print("block_hash: ", block_hash)
print("block_height: ", block_height)
print("block_timestamp: ", block_timestamp)
print("block_date: ", block_date) 





previousblockhash:  00000000000000000001c70b63927b1606e758f43fd52fa71b67caae68e8711a
nextblockhash:  00000000000000000005b0199ccf91c9d24e7ca515713db54abb665c9958f784
block_hash:  00000000000000000008bb27dd339ad1b2534f9060907a2be6925bb375395ba2
block_height:  681545
block_timestamp:  2021-05-02 16:50:00
block_date:  2021-05-02


# Transaction Input Address
The data you are looking for isn't actually included in the transaction that spends the UTXOs. Addresses are basically shorthand for a scriptPubKey, and scriptPubKeys are associated with outputs, not inputs. So if you want to check if an input spends from a certain address, you have to get the corresponding previous output and look what address it pays to.

In [125]:
# parse transactions which are contained in one block 

# belongs relationship - txid, block_hash, 'BELONGS_TO' OK => all these transactions belong to the same block
# transaction - txid, block_date, inDegree, outDegree

for tx in block['tx']: 
    print(tx, block_hash, 'BELONGS_TO')
    print("\n-----TxID: ", tx)
    command = "bitcoin-cli getrawtransaction " + tx + " true"
    stdin, stdout, stderr = ssh.exec_command(command)
    rawtx = json.loads(stdout.read().decode("utf-8"))
    print("Indegree: ", len(rawtx['vin']))
    print("Outdegree: ", len(rawtx['vout']))
    
    # ([in_address, in_value, tx_id, 'SENDS'])
    for vin in rawtx['vin']:
        try: 
            print("TxID Input: ", vin['txid'])
        except Exception as e:
            pass
        break;
    
    # receives relationship - [tx_id, val, o, addr, 'RECEIVES'] o= index array OK
    for v in rawtx['vout']:
        o = v['n']
        val = v['value']
        print("\n#Addresses: ", len(v['scriptPubKey']['addresses']))
        if len(v['scriptPubKey']['addresses'])> 0: 
            for a in v['scriptPubKey']['addresses']:
                print(a)
        break;



cebc8ac65d27d9ae05aad5249e5b991e01d50b256118ea1722373a212814b81e 00000000000000000008bb27dd339ad1b2534f9060907a2be6925bb375395ba2 BELONGS_TO

-----TxID:  cebc8ac65d27d9ae05aad5249e5b991e01d50b256118ea1722373a212814b81e
Indegree:  1
Outdegree:  4

#Addresses:  1
1EepjXgvWUoRyNvuLSAxjiqZ1QqKGDANLW
241f76688bb13889f8e11b232fa3aaedd7a0a1947829f821ea2d4655705d54c0 00000000000000000008bb27dd339ad1b2534f9060907a2be6925bb375395ba2 BELONGS_TO

-----TxID:  241f76688bb13889f8e11b232fa3aaedd7a0a1947829f821ea2d4655705d54c0
Indegree:  1
Outdegree:  9
TxID Input:  69bbf59972bd6c9ad9fefca9f3ef22122d76e7eb1382f62b297593d964ba25b7

#Addresses:  1
3EbnuEVF7RxgbY8W87sUetP74KZZCFeSe2
e370b0551e4ef513ade2dd9ba219be5c815643d24c99acdb09af210df121cae9 00000000000000000008bb27dd339ad1b2534f9060907a2be6925bb375395ba2 BELONGS_TO

-----TxID:  e370b0551e4ef513ade2dd9ba219be5c815643d24c99acdb09af210df121cae9
Indegree:  2
Outdegree:  2
TxID Input:  ed8950fb1d5221801e6b21dcdf1e4ab0a4195310b1eabadff9ecab55cfbbf56a

#Ad

Indegree:  2
Outdegree:  3
TxID Input:  3ed01ce82920ecb07747b8bb3563df6572a00afd8dbc8cd2ac3ed420e15e4244

#Addresses:  1
bc1qpqd6gc70x7cjfye56w3tdh39mdzkk80agta9s7nkl9x5gr8ggtpqzc6us9
a703b4f389b537276e243121a2c29425521bdbcce9b84041a107e76085b6e9eb 00000000000000000008bb27dd339ad1b2534f9060907a2be6925bb375395ba2 BELONGS_TO

-----TxID:  a703b4f389b537276e243121a2c29425521bdbcce9b84041a107e76085b6e9eb
Indegree:  1
Outdegree:  3
TxID Input:  c188233ca69fe9c115608569c1edba99b2e55608bbbc030675e4d543384ce5a4

#Addresses:  1
bc1q48nn9xw865s7wthtzxjdv4mmsffuhdwl6e5ey6x7l5ts8dfz0adqwnlhq0
100facbfb42062a05f19435202a8439fb318b0e022bbe4e9c99c8c0556df90ee 00000000000000000008bb27dd339ad1b2534f9060907a2be6925bb375395ba2 BELONGS_TO

-----TxID:  100facbfb42062a05f19435202a8439fb318b0e022bbe4e9c99c8c0556df90ee
Indegree:  1
Outdegree:  2
TxID Input:  91ee26aeddd6b16def65a686147494945159b0485a8f396aaf46fb2775bcda84

#Addresses:  1
3FW8mPNvsaMCibL93Jz2Uh8UoVY1fn25pX
7c62fcc602aeb5ffbca68f5929856dad079c99

79b9892213422a8270ea5a21320a7830c679f2686fb966529dc2c89542b2a8b8 00000000000000000008bb27dd339ad1b2534f9060907a2be6925bb375395ba2 BELONGS_TO

-----TxID:  79b9892213422a8270ea5a21320a7830c679f2686fb966529dc2c89542b2a8b8
Indegree:  1
Outdegree:  1
TxID Input:  f2a92deb31f547634b95926de05554ef52373c5c3a46d3fe0d6c335fb51d7113

#Addresses:  1
1CN7GBwgfHwzgdzWFk3N9h1GF4PZ4rv6rb
e6f0eed0ba2370effd004f6ca688db38baaed131e19dc4f164d21ef4ba065044 00000000000000000008bb27dd339ad1b2534f9060907a2be6925bb375395ba2 BELONGS_TO

-----TxID:  e6f0eed0ba2370effd004f6ca688db38baaed131e19dc4f164d21ef4ba065044
Indegree:  1
Outdegree:  1
TxID Input:  613250ef3671d028beac5eb594a3b263b9fca29f99a99ed0b527b981d1f2fb18

#Addresses:  1
bc1qf0f5qvdc7a9tl359g2gfezs7034tgp7xn42ety
269e2799e886da9708598170b577e39700e835f45a2a6a77462620ab8b870fbd 00000000000000000008bb27dd339ad1b2534f9060907a2be6925bb375395ba2 BELONGS_TO

-----TxID:  269e2799e886da9708598170b577e39700e835f45a2a6a77462620ab8b870fbd
Indegree:  1
Outdegree:

Indegree:  1
Outdegree:  2
TxID Input:  22a1b41d1eb421140104d70918f8318b7d75dac52999b0ea8564e1290525463a

#Addresses:  1
3DV24sXgBDBecqMnoGsMqnGrN6RA4ZqMfw
ace5c6496fc1c61aaa074f976181525a0e7464637a6e09506fcfbe33c6df8ddd 00000000000000000008bb27dd339ad1b2534f9060907a2be6925bb375395ba2 BELONGS_TO

-----TxID:  ace5c6496fc1c61aaa074f976181525a0e7464637a6e09506fcfbe33c6df8ddd
Indegree:  1
Outdegree:  2
TxID Input:  96030d0d2b0eaeb4ebc3c5a0d644b23fe284526914192f5b4436ef64aeb2e061

#Addresses:  1
3HHWkNCPqH795ogWSQeA1YktXDEnaBnSX1
fa2c5354a269d11b0cc305036fc2fbf11d2adcdaa561b673852afed6b6365ade 00000000000000000008bb27dd339ad1b2534f9060907a2be6925bb375395ba2 BELONGS_TO

-----TxID:  fa2c5354a269d11b0cc305036fc2fbf11d2adcdaa561b673852afed6b6365ade
Indegree:  1
Outdegree:  2
TxID Input:  583675106f3fed1c8d5f51545089e0b2f9619b3a6a37d870a0423db133e12b47

#Addresses:  1
3LHQuKscrRsVZM1YSzs7nvGvJaNYkt99Qe
2cb5f5984dc33c568539a105af5e32f9f9751d2eb0b57d304ef4489aa73d1ae7 00000000000000000008bb27dd339


#Addresses:  1
36F3WCCaqZhh6eMUGjQVZMpHwbatDK9aXZ
c448361efe195ae660f9943368d712f26ac02fa5879244113eb3742395382deb 00000000000000000008bb27dd339ad1b2534f9060907a2be6925bb375395ba2 BELONGS_TO

-----TxID:  c448361efe195ae660f9943368d712f26ac02fa5879244113eb3742395382deb
Indegree:  2
Outdegree:  1
TxID Input:  d0f573a14ded7a2246b9746c9ea4a5143146d9c61f9a44b665b698706e467cfa

#Addresses:  1
3B4xsnS692iVaGPfvFAZCopV5TaLzJi2sb
0f0d8cdbb210e4d2e02deefc5b6b1054f929e33b75e4bd470a99b33ea0110ff7 00000000000000000008bb27dd339ad1b2534f9060907a2be6925bb375395ba2 BELONGS_TO

-----TxID:  0f0d8cdbb210e4d2e02deefc5b6b1054f929e33b75e4bd470a99b33ea0110ff7
Indegree:  1
Outdegree:  2
TxID Input:  5baeb1b08971f3b9a28cff95c95bca0b44e032717f10a15d165acb94f8300ab6

#Addresses:  1
1GXv8B4FYZGoMnCUHJa56Q94Z9jeyFQsyu
6ade877c368affe70e79d0968863bd183a2a5c18a625f10a52a780ae42e19348 00000000000000000008bb27dd339ad1b2534f9060907a2be6925bb375395ba2 BELONGS_TO

-----TxID:  6ade877c368affe70e79d0968863bd183a2a5c18a625f

KeyError: 'addresses'